In [15]:
import pyspark
print(pyspark.__version__)
import sys
print(sys.version)
import yt_dlp
print(yt_dlp.version.__version__)
import cv2
print(cv2.__version__)

3.3.2
3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC 11.3.0]
2023.02.17
4.7.0


In [93]:
#!mamba update --yes mamba
#!mamba update --yes youtube-dl
!mamba install --yes google-cloud-storage
!mamba install --yes google-api-python-client
!mamba install --yes google-cloud-videointelligence
!mamba install --yes openai


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.3.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['google-cloud-storage']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━━╸━━━━━━━━━━━━━━━   0.0 B /  

In [47]:
!youtube-dl --version

2021.12.17


In [48]:
import os, re
import json
import fnmatch
import subprocess
from pydub import AudioSegment  # to extract as podcast
from youtube_dl import YoutubeDL
#from credentials.config import google_api_key
from nltk import word_tokenize
from nltk.util import ngrams
import numpy as np
from google.cloud import storage
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def to_milliseconds(timestamp):
    hours, minutes, seconds = map(int, timestamp.split('.')[0].split(':'))
    milliseconds = int(timestamp.split(' ')[0].split('.')[-1])
    return (hours * 60 * 60 + minutes * 60 + seconds) * 1000 + milliseconds

def from_milliseconds(milliseconds):
    hours = milliseconds // (60 * 60 * 1000)
    milliseconds = milliseconds % (60 * 60 * 1000)
    minutes = milliseconds // (60 * 1000)
    milliseconds = milliseconds % (60 * 1000)
    seconds = milliseconds // 1000
    milliseconds = milliseconds % 1000
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"
    
def cosine_similarity(text1, text2):
    tfidf = TfidfVectorizer().fit_transform([text1, text2])
    return cosine_similarity(tfidf)[0][1]

def aggregate_texts(start_time, end_time, text, next_start_time, next_end_time, next_text):
    similarity = cosine_similarity(text, next_text)
    if similarity > 0.5:
        aggregated_text = text + " " + next_text
        aggregated_start_time = start_time
        aggregated_end_time = next_end_time
        return aggregated_start_time, aggregated_end_time, aggregated_text, True
    else:
        return start_time, end_time, text, False


In [50]:
def vtt_to_json(vtt_file_path:str)->str:
    """ Call the function to convert a .vtt file to a .json file
    vtt_to_json('path/to/file.vtt')
    """
    # Open the .vtt file and read its contents
    with open(vtt_file_path, 'r') as f:
        vtt_data = f.read()

    # Split the VTT data into lines
    lines = vtt_data.split("\n")

    # Skip the first three lines (WEBVTT, Kind, Language)
    lines = lines[3:]

    # Initialize an empty list to store the captions
    captions = []

    # Loop through the remaining lines
    i = 0
    while i < len(lines):
        # Extract the start and end times
        times = re.findall(r'\d\d:\d\d:\d\d.\d\d\d', lines[i])
        if times:
            if len(times) > 1:
                start_time = times[0]
                end_time = times[1]
            else: # link to a diff video witha timestamp
                start_time = times[0]
                end_time = times[0]

            # Initialize an empty list to store the caption text
            caption_text = []

            # Move to the next line
            i += 1

            # Continue until we reach a line with just times
            while i < len(lines) and re.search(r'\d\d:\d\d:\d\d.\d\d\d', lines[i]) is None:
                caption_text.append(lines[i].strip())
                i += 1

            # Join the lines of caption text
            caption = " ".join(caption_text)

            # Add the start time, end time, and caption text to the captions list
            if caption and caption != " ":
                captions.append({
                    "start_time": start_time,
                    "end_time": end_time,
                    "caption": caption.strip()
                })
        else:
            i += 1

    # Convert the captions list to JSON
    captions_json = json.dumps(captions, indent=4)

    # Save the JSON data to a file
    json_file_path = vtt_file_path.replace(".vtt", ".json")
    with open(json_file_path, 'w') as f:
        f.write(captions_json)

    return json_file_path


In [86]:
def upload_to_gcs(path2file: str, fnames: list, bucket_name: str, prefix: str = '') -> list:
    """Upload the video or subtitle files to the specified GCS bucket"""
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        object_names = []
        for fname in fnames:
            if prefix:
                object_name = f"{prefix}/{fname}"
            else:
                object_name = fname

            blob = bucket.blob(object_name)
            if path2file:
                blob.upload_from_filename(f"{path2file}/{fname}")
            else:
                blob.upload_from_filename(fname)

            object_names.append(f"gs://{bucket_name}/{object_name}")

        return object_names
    except Exception as e:
        return [f"Error uploading {fname}: {e}" for fname in fnames]

def fetch_files_locally(directory:str, extension:str='vtt')->list:
    """ finds all files with specified extension in supplied directory """
    ffiles = []
    for file in os.listdir(directory):
        if fnmatch.fnmatch(file, f'*.{extension}'):
            ffiles.append(file)
    return ffiles

In [87]:
links = [   #pipeRider
            "https://www.youtube.com/watch?v=O-tyUOQccSs",
            # Kafka+flink
            "https://www.youtube.com/watch?v=bG2U7c5rYBo",
            # zoomcamp playlist
            "https://www.youtube.com/watch?v=-zpVha7bw5A&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=1",
            "https://www.youtube.com/watch?v=W3Zm6rjOq70&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=17"
]

#yt = YoutubeDL(params={'verbose': True}).extract_info(url="https://www.youtube.com/watch?v=bG2U7c5rYBo", download=False)
#title = yt.get("title")
#yt = YoutubeDL().extract_info(url=links[0], download=False)
#title = yt.get("title")
#description = yt.get('description')
#print(title)
#print(description)
bucket ='sparkhudi'
prefix = 'https://www.youtube.com/watch?v=-zpVha7bw5A&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=1'.split('_')[-1].split('&index=')[0]
lbl = 'extra'
path2file = "videos/" + lbl

In [88]:
# cut out a part of the video
#!ffmpeg -i input.mp4 -ss 00:01:30 -to 00:02:30 -c copy output.mp4
def download_video(link: str, path2file: str, bucket_name: str, prefix: str = '') -> list:
    """Download the video in mp4 format with 720p resolution and subtitle files in .vtt format to the specified GCS bucket."""
    #try:
        # Download the video and subtitle files to the local machine
    cmdcaps = [
        "yt-dlp", "-N", "16", "--format", "bv[height=720][ext=mp4]+ba[ext=m4a]",
        "--write-subs", "--write-auto-subs", "--sub-format", "ass/srt/best", "--sub-langs", "en",
        "-o", f"{path2file}/%(title)s.%(ext)s", link
    ]
    result = subprocess.run(cmdcaps, capture_output=True, text=True)
    output_lines = result.stdout.strip().split('\n')
    
    for line in output_lines:
        if '[download]' in line and 'Destination' in line:
            filename = line.split('Destination: ')[-1].strip()
        elif '[info]' in line and 'Writing video subtitles to:' in line:
            sub_filename = line.split('Writing video subtitles to: ')[-1].strip()
    
    return filename.replace('.en.vtt','.mp4'), sub_filename.replace('.mp4', '')


In [89]:
path2file = 'videos/extra'
fname, subfname = download_video("https://www.youtube.com/watch?v=O-tyUOQccSs", path2file, 'sparkhudi')
jsonfname = vtt_to_json(subfname)
# filenames contain path
stats = upload_to_gcs('', [fname, subfname, jsonfname], bucket, 'extra')
print(stats)

['gs://sparkhudi/extra/videos/extra/Maximizing Confidence in Your Data Model Changes with dbt and PipeRider - Dave Flynn.mp4', 'gs://sparkhudi/extra/videos/extra/Maximizing Confidence in Your Data Model Changes with dbt and PipeRider - Dave Flynn.en.vtt', 'gs://sparkhudi/extra/videos/extra/Maximizing Confidence in Your Data Model Changes with dbt and PipeRider - Dave Flynn.en.json']


In [95]:
from search import autolabel_frames
labels = autolabel_frames(stats[0])
print(labels)


NameError: name 'videointelligence' is not defined

In [ ]:
def fetch_files_from_bucket(bucket_name:str, suffix:str) -> list:
    """Fetch files from a GCS bucket and subfolder."""
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name + "/" + suffix)
        file_list = [blob.name for blob in bucket.list_blobs() ]

        return file_list

    except Exception as e:
        return e

In [ ]:
def bulk_upload(path2file: str, bucket_name: str):
    """ Upload locally stored files to GCS bucket with subdirectories based on file type. """
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)

        # Create subdirectories
        blob_mp4 = bucket.blob('mp4/')
        blob_mp4.upload_from_string('')
        blob_vtt = bucket.blob('vtt/')
        blob_vtt.upload_from_string('')
        blob_json = bucket.blob('json/')
        blob_json.upload_from_string('')

        # Upload files
        for filename in os.listdir(path2file):
            if filename.endswith('.mp4'):
                blob = bucket.blob('mp4/' + filename)
                blob.upload_from_filename(path2file + '/' + filename)
            elif filename.endswith('.vtt'):
                blob = bucket.blob('vtt/' + filename)
                blob.upload_from_filename(path2file + '/' + filename)
            elif filename.endswith('.json'):
                blob = bucket.blob('json/' + filename)
                blob.upload_from_filename(path2file + '/' + filename)

        return True

    except Exception as e:
        return e